## Ordnance Survey Data Hub Tutorials - Example Applications

# Using the OS Maps API with ipyleaflet (magnifying glass)

This is a simple Python/Jupyter Notebook application, demonstrating how the OS Maps API can be used to display two styles of background mapping within an interactive ipyleaflet (https://ipyleaflet.readthedocs.io/en/latest/) map.  

The OS Maps API, is one of the many services available from the OS Data Hub.  It's a service based upon the WMTS and RESTful ZXY mapping standards, that can be used, within leaflet, as a source for background contextual mapping, or as the primary feature of interest. 

This demo shows two of the standard OS Maps API styles being used together, via ipyleaflet's 'magnifying glass' feature.

In [1]:
from datetime import datetime
from ipyleaflet import Map, MagnifyingGlass, basemap_to_tiles, Marker, Icon, WidgetControl
from urllib.parse import unquote, urlencode
from ipywidgets import Image
import IPython

Use of the OS APIs requires a service key. Sign up for yours at https://osdatahub.os.uk/ and enter it below.

In [2]:
# OS Data Hub project API key (key from guy.heathcote@os.uk)
key = 'YOUR API KEY HERE'

The following cells set up a basic ipyleaflet map object, configured to use the OS Maps API. It will be initially centred on central London, with a zoom level of 15.

In [3]:
# Function to create ipyleaflet tile layers
def createTileLayer(layertype):

    params = urlencode({'key': key,
                        'service': 'WMTS',
                        'request': 'GetTile',
                        'version': '2.0.0',
                        'height': 256,
                        'width': 256,
                        'outputFormat': 'image/png',
                        'style': 'default',
                        'layer': layertype,
                        'tileMatrixSet': 'EPSG:3857',
                        'tileMatrix': '{z}',
                        'tileRow': '{y}',
                        'tileCol': '{x}'})

    # OS Data Hub base path - https://api.os.uk
    # OS Maps API WMTS end point path - /maps/raster/v1/wmts?
    url = unquote(f'https://api.os.uk/maps/raster/v1/wmts?{params}')

    tileLayer = {'url': url,
               'min_zoom': 7,
               'max_zoom': 20,
               'attribution': f'Contains OS data &copy; Crown copyright and database rights {datetime.now().year}'}

    return tileLayer

In [4]:
# Create custom ipyleaflet TileLayers for the OS Maps API WMTS resource
os_maps_api_light = createTileLayer('Light_3857')
os_maps_api_road = createTileLayer('Road_3857')

In [5]:
# Create ipyleaflet Map
centreOfMap = [51.507, -0.105]  # Central London
m = Map(basemap=os_maps_api_light,
        center=centreOfMap,
        zoom=15)  # Initial zoom level = 12

Create and apply the magnified layer.

The ipyleaflet magnifying glass can be confirgured to work in various ways.  Here, we'll increase it's radius to 150 (default=100) and set the zoom offset to 2 (default=3).  A full list of options can be found at https://ipyleaflet.readthedocs.io/en/latest/api_reference/magnifying_glass.html.

In [6]:
# Add and configure the magnified layer
road_layer = basemap_to_tiles(os_maps_api_road)
magnifying_glass = MagnifyingGlass(radius=150, zoom_offset=2, layers=[road_layer])

m.add_layer(magnifying_glass)

In [7]:
# Add an image overlay, placing the OS logo in the bottom left corner.
oslogo_url = 'https://raw.githubusercontent.com/OrdnanceSurvey/os-api-branding/master/img/os-logo-maps.png'
oslogo_img = IPython.display.Image(oslogo_url, width = 300)
oslogoWidget = Image(value=oslogo_img.data, format='png', width=100, height=30)
widgetControl = WidgetControl(widget=oslogoWidget, position='bottomleft')
m.add_control(widgetControl)


In [8]:
# Show the map
m

Map(center=[51.507, -0.105], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…